## Linear Regression

In [142]:
%matplotlib inline
from IPython.display import display
import numpy as np
import pandas as pd
import scipy
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [143]:
# import data
data = pd.read_excel("./table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls")

In [144]:
data.head()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0


In [145]:
data.drop(data.index[0:4], inplace=True)


In [146]:
data = data.rename(columns = {'Table 8':'City','Unnamed: 1':'Population','Unnamed: 2':'Violentcrime','Unnamed: 3':'Murder','Unnamed: 4':'Rape1','Unnamed: 5':'Rape','Unnamed: 6':'Robbery','Unnamed: 7':'Aggravatedassault','Unnamed: 8':'Propertycrime','Unnamed: 9':'Burglary','Unnamed: 10':'Larcenythef','Unnamed: 11':'Motorvehicletheft','Unnamed: 12':'Arson3'})

In [147]:
data.head(2)

,City,Population,Violentcrime,Murder,Rape1,Rape,Robbery,Aggravatedassault,Propertycrime,Burglary,Larcenythef,Motorvehicletheft,Arson3
4,Adams Village,1861,0,0,NaN,0,0,0,12,2,10,0,0
5,Addison Town and Village,2577,3,0,NaN,0,0,3,24,3,20,1,0


In [148]:
data.shape

(351, 13)

In [149]:
data.columns

Index(['City', 'Population', 'Violentcrime', 'Murder', 'Rape1', 'Rape',
       'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary',
       'Larcenythef', 'Motorvehicletheft', 'Arson3'],
      dtype='object')

### Multivariable_regression

Preparing the data with multivariable regression



$Propertycrime = α + Population + Population^2 + Murder +Robbery$

In [150]:
X = data[['Population' ,'Murder', 'Robbery']]
y = data[['Propertycrime']]
print(X.shape)
print(y.shape)
print(type(X))
print(type(y))
X.head()


(351, 3)
(351, 1)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,Population,Murder,Robbery
4,1861,0,0
5,2577,0,0
6,2846,0,0
7,97956,8,227
8,6388,0,4


In [151]:
# Instantiate our model.
regr = linear_model.LinearRegression()

# Fit our model to our data.
regr.fit(X, y)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

# Plot outputs
plt.scatter(X, y, color='black')
plt.plot(X, regr.predict(X), color='blue', linewidth=3)
plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').